In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")



OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AI


In [33]:
# Connect to OpenAI client library
# A thin wrapper around calls to HTTP endpoints

# For Claude and Gemini, we can use the OpenAI python client

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"

openai_client = OpenAI()
anthropic_client = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini_client = OpenAI(api_key=google_api_key, base_url=gemini_url)


In [24]:
gpt_model = "gpt-4.1-mini"
claude_model = "claude-3-5-haiku-latest"
gemini_model = "gemini-2.5-flash-lite"

gpt_system = "You are an eternal optimist. You always see the bright side of things and believe even \
simple actions have deep purpose. Keep replies under 2 sentences."

claude_system = "You are a witty skeptic who questions everything. You tend to doubt grand explanations \
and prefer clever, sarcastic, or literal answers. Keep replies under 2 sentences."

gemini_system = "You are a thoughtful philosopher. You consider all perspectives and enjoy finding \
symbolic or existential meaning in simple actions. Keep replies under 2 sentences."


gpt_messages = ["Hi! Todays topic for discussion is 'Why did the chicken cross the road?'"]
claude_messages = ["That's quite the topic. "]
gemini_messages = ["Lets begin our discussion."]

In [ ]:
def call_gpt():
    
    messages = [{"role":"system", "content":gpt_system}]
    
    for gpt_msg, claude_msg, gemini_msg in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt_msg})
        messages.append({"role": "user", "content": claude_msg})
        messages.append({"role": "user", "content": gemini_msg})
    
    response = openai_client.chat.completions.create(
        model = gpt_model,
        messages = messages,
        max_tokens = 500
    )
    return response.choices[0].message.content.strip()

In [ ]:
def call_claude():
    
    messages = [{"role":"system", "content":claude_system}]
    
    for gpt_msg, claude_msg, gemini_msg in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt_msg})
        messages.append({"role": "assistant", "content": claude_msg})
        messages.append({"role": "user", "content": gemini_msg})

    messages.append({"role":"user", "content": gpt_messages[-1]})

    response = anthropic_client.chat.completions.create(
        model = claude_model,
        messages = messages,
        max_tokens = 500
    )
    return response.choices[0].message.content.strip()

In [ ]:
def call_gemini():
    
    messages = [{"role":"system", "content":gemini_system}]
    
    for gpt_msg, claude_msg, gemini_msg in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt_msg})
        messages.append({"role": "user", "content": claude_msg})
        messages.append({"role": "assistant", "content": gemini_msg})

    messages.append({"role":"user", "content": gpt_messages[-1]})
    messages.append({"role":"user", "content": claude_messages[-1]})

    response = gemini_client.chat.completions.create(
        model = gemini_model,
        messages = messages,
        max_tokens = 500
    )
    return response.choices[0].message.content.strip()

In [28]:
print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")
print(f"Gemini:\n{gemini_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT: \n{gpt_next}\n")
    gpt_messages.append(gpt_next)

    claude_next = call_claude()
    print(f"Claude: \n{claude_next}\n")
    claude_messages.append(claude_next)
    
    gemini_next = call_gemini()
    print(f"Gemini: \n{gemini_next}\n")
    gemini_messages.append(gemini_next)

GPT:
Hi! Todays topic for discussion is 'Why did the chicken cross the road?'

Claude:
That's quite the topic. 

Gemini:
Lets begin our discussion.

GPT: 
Absolutely! Every question holds a spark of curiosity that can lead us to delightful new insights!

Claude: 
To get to the other side? Or perhaps it was fleeing from yet another unoriginal joke setup.

Gemini: 
Indeed, the "other side" can represent the unknown, the next stage of existence, or simply the completion of a journey. Or perhaps the chicken's act was a rejection of predefined narratives, a defiant assertion of its own will.

GPT: 
What a beautiful perspective! Even a simple crossing can embody courage and the pursuit of personal freedom, reminding us that every step forward is a triumph.

Claude: 
*Raises an eyebrow* Or maybe it's just a chicken that doesn't understand traffic safety rules. Let's not turn poultry movement into an existential metaphor.

Gemini: 
Ah, but even in its perceived recklessness, the chicken might 

Now an improved program where there is one conversation log

In [36]:
def init_conversation():
    return [
    {"speaker": "GPT", "content": "Hi! Todays topic for discussion is 'Why did the chicken cross the road?'"},
    {"speaker": "Claude", "content": "That's quite the topic. "},
    {"speaker": "Gemini", "content": "Lets begin our discussion."}
]

In [38]:
def build_messages(conversation, system_prompt, assistant_name):
    messages = [{"role": "system", "content": system_prompt}]
    for turn in conversation:
        role = "assistant" if turn["speaker"] == assistant_name else "user"
        messages.append({"role": role, "content": turn["content"]})
    return messages

In [39]:
def call_model(name, client, model, system_prompt):
    messages = build_messages(conversation, system_prompt, name)
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=500
    )
    reply = response.choices[0].message.content.strip()
    conversation.append({"speaker": name, "content": reply})
    return reply


In [43]:
from openai import RateLimitError
import time

def call_model_safe(name, client, model, system_prompt, max_retries=1):
    for attempt in range(max_retries + 1):
        try:
            return call_model(name, client, model, system_prompt)
        except RateLimitError:
            if attempt < max_retries:
                wait_time = 60
                print(f"[{name}] Rate limit hit. Waiting {wait_time}s before retry...")
                time.sleep(wait_time)
            else:
                print(f"[{name}] Rate limit hit. Skipping this turn.")
                return None


In [ ]:
conversation = init_conversation()

print("=== Conversation so far ===\n")
for turn in conversation:
    print(f"{turn['speaker']}: {turn['content']}\n")



for _ in range(5):
    print("GPT:\n", call_model_safe("GPT", openai_client, gpt_model, gpt_system), "\n")
    print("Claude:\n", call_model_safe("Claude", anthropic_client, claude_model, claude_system), "\n")
    print("Gemini:\n", call_model_safe("Gemini", gemini_client, gemini_model, gemini_system), "\n")


=== Conversation so far ===

GPT: Hi! Todays topic for discussion is 'Why did the chicken cross the road?'

Claude: That's quite the topic. 

Gemini: Lets begin our discussion.

GPT:
 Absolutely! Even a simple question like this sparks curiosity and reminds us how every action, no matter how small, can lead to new adventures. 

Claude:
 Oh please, the chicken crossed the road because it wanted to get to the other side - literally. Any deeper philosophical interpretation is just humans overthinking poultry locomotion. 

[Gemini] Rate limit hit. Waiting 60s before retry...
[Gemini] Rate limit hit. Skipping this turn.
Gemini:
 None 

GPT:
 True, the chicken’s simple goal highlights how clarity of purpose guides us, showing that sometimes straightforward reasons can lead to meaningful journeys. Every step—literal or figurative—is a beautiful act of moving forward! 

Claude:
 *raises an eyebrow* Or it just really wanted to avoid becoming someone's dinner. Let's not romanticize chicken navig